In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keplerutils
import seaborn as sns

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
df = pd.read_csv('final.csv', index_col = 0)

In [9]:
df.head()

,disposition,period,time0bk,impact,duration,depth,prad,teq,insol,model_snr,steff,srad,ra,dec,kepmag,slogg_exp
0,1,9.488036,170.538750,0.146,2.95750,615.8,2.26,793.0,93.59,35.8,5455.0,0.927,291.93423,48.141651,15.347,87.095046
1,1,54.418383,162.513840,0.586,4.50700,874.8,2.83,443.0,9.11,25.8,5455.0,0.927,291.93423,48.141651,15.347,87.095046
2,0,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638.0,39.30,76.3,5853.0,0.868,297.00482,48.134129,15.436,94.066314
3,0,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395.0,891.96,505.6,5805.0,0.791,285.53461,48.285210,15.597,95.966579
4,1,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406.0,926.16,40.9,6031.0,1.046,288.75488,48.226200,15.509,84.605561


Features sorted by their score:
[(0.2567, 'prad'), (0.2048, 'model_snr'), (0.1122, 'period'), (0.0871, 'impact'), (0.0664, 'duration'), (0.0508, 'teq'), (0.0407, 'depth'), (0.0397, 'insol'), (0.0257, 'time0bk'), (0.024, 'ra'), (0.0234, 'steff'), (0.0214, 'dec'), (0.0169, 'srad'), (0.0168, 'slogg_exp'), (0.0136, 'kepmag')]

In [65]:
df_test = df.drop(labels=['kepmag', 'slogg_exp', 'srad', 'dec', 'steff', 'ra', 'time0bk'], axis=1)

In [68]:
X_train_res2, X_test2, y_train_res2, y_test2 = keplerutils.split_and_upsample(df_test)

In [14]:
X_train_res, X_test, y_train_res, y_test = keplerutils.split_and_upsample(df)

# Compare final models

## Logistic Regression

In [79]:
def logisticCV(X_train, y_train):
    '''
        Performs logistic regression with 5-fold cross validated grid search on training data.
        Prints AUC scores and best parameters.
       ----Parameters----
        X_train_res: SMOTE-resampled features
        y_train_res: SMOTE-resampled response
        '''
    X_train_res, X_test, y_train_res, y_test = keplerutils.split_and_upsample(df)
    
    pipe = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', max_iter=1000))
            
    tuning_params = {'logisticregression__C': [1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7]}
    
    g = GridSearchCV(pipe, tuning_params, scoring='accuracy', cv=5, n_jobs = -1)
    
    g.fit(X_train, y_train)
    
    y_preds = g.predict(X_test2)
    
    print(classification_report(y_test2, y_preds))
            
    print('Best parameters: ', g.best_params_)
    
    print('Mean ROC grid scores: ', g.cv_results_['mean_test_score'])

In [80]:
logisticCV(X_train_res, y_train_res)

ValueError: operands could not be broadcast together with shapes (1703,8) (15,) (1703,8) 

## Random Forest

In [86]:
def RFCCV(X_train, y_train):
    '''
        Performs Random Forest Classificiation with 5 fold cross validation on training data.
        Prints ROC and and best parameters.
        ----Parameters----
        X_train_res: SMOTE-resampled features
        y_train_res: SMOTE-resampled response
    '''                
    pipe = make_pipeline(StandardScaler(), RandomForestClassifier())
        
    tuning_params = {'randomforestclassifier__max_depth': [50, 100, 150, 200, 250, 300], 
                     'randomforestclassifier__n_estimators': [50, 100, 150, 200, 250, 300],
                     'randomforestclassifier__max_features': [2, 4, 6, 8]
                    }
    
    g = RandomizedSearchCV(pipe, tuning_params, scoring='recall_micro', cv=5, n_jobs = -1)
    
    g.fit(X_train, y_train)
    
    y_preds = g.predict(X_test2)
    
    print("Features sorted by their score:")
    
    print(sorted(zip(map(lambda x: round(x, 4), g.best_estimator_.steps[1][1].feature_importances_), 
                    df.iloc[:, 1:].columns.values), reverse=True))
    
    print()
    
    print(classification_report(y_test2, y_preds))
            
    print('Best parameters: ', g.best_params_)
    
    print('Mean ROC grid scores: ', g.cv_results_['mean_test_score'])

## Gradient Boosting

In [81]:
def GBCV(X_train, y_train):
    pipe = make_pipeline(StandardScaler(), GradientBoostingClassifier())
        
    tuning_params = {'gradientboostingclassifier__learning_rate': [1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3],
                     'gradientboostingclassifier__max_depth': [50, 100, 150, 200],
                     'gradientboostingclassifier__max_features': [2, 4, 6, 8]}
    
    g = RandomizedSearchCV(pipe, tuning_params, cv=5, scoring='recall_micro')
    
    g.fit(X_train, y_train)
    
    y_preds = g.predict(X_test2)
    
    print(classification_report(y_test2, y_preds))
            
    print('Best parameters: ', g.best_params_)
    
    print('Mean ROC grid scores: ', g.cv_results_['mean_test_score'])

## AdaBoost

In [82]:
def ABCV(X_train, y_train):
    pipe = make_pipeline(StandardScaler(), AdaBoostClassifier())
        
    tuning_params = {'adaboostclassifier__learning_rate': [1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3],
                     'adaboostclassifier__n_estimators': [50, 100, 150, 200]}
    
    g = RandomizedSearchCV(pipe, tuning_params, cv=5, scoring='recall_micro')
    
    g.fit(X_train, y_train)
    
    y_preds = g.predict(X_test2)
    
    print(classification_report(y_test2, y_preds))
            
    print('Best parameters: ', g.best_params_)
    
    print('Mean ROC grid scores: ', g.cv_results_['mean_test_score'])

## Comparison of the models

In [83]:
GBCV(X_train_res2, y_train_res2)

              precision    recall  f1-score   support

           0       0.95      0.91      0.93      1131
           1       0.84      0.91      0.87       572

   micro avg       0.91      0.91      0.91      1703
   macro avg       0.90      0.91      0.90      1703
weighted avg       0.91      0.91      0.91      1703

Best parameters:  {'gradientboostingclassifier__max_features': 2, 'gradientboostingclassifier__max_depth': 150, 'gradientboostingclassifier__learning_rate': 0.01}
Mean ROC grid scores:  [0.93103448 0.93015031 0.89404657 0.9360448  0.94017094 0.92764515
 0.90480401 0.90583554 0.89832007 0.90789861]


In [90]:
import pickle

In [87]:
RFCCV(X_train_res2, y_train_res2)

Features sorted by their score:
[(0.2596, 'depth'), (0.2182, 'insol'), (0.1273, 'period'), (0.1161, 'time0bk'), (0.0854, 'impact'), (0.0735, 'prad'), (0.0662, 'duration'), (0.0536, 'teq')]

              precision    recall  f1-score   support

           0       0.96      0.91      0.93      1131
           1       0.83      0.93      0.88       572

   micro avg       0.91      0.91      0.91      1703
   macro avg       0.90      0.92      0.91      1703
weighted avg       0.92      0.91      0.91      1703

Best parameters:  {'randomforestclassifier__n_estimators': 200, 'randomforestclassifier__max_features': 4, 'randomforestclassifier__max_depth': 250}
Mean ROC grid scores:  [0.93692897 0.93398173 0.93707633 0.93678161 0.93737106 0.93442381
 0.93692897 0.93678161 0.93545535 0.93766578]


In [ ]:
pickle.dump

In [88]:
logisticCV(X_train_res2, y_train_res2)

              precision    recall  f1-score   support

           0       0.94      0.78      0.85      1131
           1       0.67      0.90      0.77       572

   micro avg       0.82      0.82      0.82      1703
   macro avg       0.81      0.84      0.81      1703
weighted avg       0.85      0.82      0.82      1703

Best parameters:  {'logisticregression__C': 100000.0}
Mean ROC grid scores:  [0.80238727 0.81992337 0.83451223 0.8377542  0.83966991 0.84453286
 0.8443855  0.8455644  0.84541704 0.84541704]


In [89]:
ABCV(X_train_res2, y_train_res2)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:164: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:604: RuntimeWarning: invalid value encountered in greater
  return self.classes_.take(pred > 0, axis=0)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:164: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/py

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble

...

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:531: RuntimeWarning: invalid value encountered in greater
  ((sample_weight > 0) |
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:532: RuntimeWarning: invalid value encountered in less
  (estimator_weight < 0)))
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:604: RuntimeWarning: invalid value encountered in greater
  return self.classes_.take(pred > 0, axis=0)


              precision    recall  f1-score   support

           0       0.95      0.86      0.91      1131
           1       0.77      0.92      0.84       572

   micro avg       0.88      0.88      0.88      1703
   macro avg       0.86      0.89      0.87      1703
weighted avg       0.89      0.88      0.88      1703

Best parameters:  {'adaboostclassifier__n_estimators': 200, 'adaboostclassifier__learning_rate': 0.1}
Mean ROC grid scores:  [0.73239022 0.5        0.26731506 0.5        0.26731506 0.5
 0.90583554 0.5        0.5        0.5       ]
